# Introduction

### About Retrieval Augmented Generation
Retrieval Augmented Generation (RAG) is a versatile pattern that can unlock a number of use cases requiring factual recall of information, such as querying a knowledge base in natural language.

In its simplest form, RAG requires these steps:

1. Extract knowledge base passages from documents (once)
2. Create vector embedding representations of each passage in the knowledge base
3. Retreive question from end user and generate vector embedding for it.
4. Retrieve relevant passage(s) from knowledge base (for every user query) using vector similarity search
5. Generate a response by feeding retrieved passage into a large language model (for every user query)

You will work through each of these steps in the following notebook.

<p align="center">
  <img src="images/rag-architecture-basic.png" width="600"/>
</p>

### Embeddings and Vector Databases
The current state-of-the-art in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.

We can generate dense vector representations using embedding models. In this notebook, we use [SentenceTransformers](https://www.google.com/search?client=safari&rls=en&q=sentencetransformers&ie=UTF-8&oe=UTF-8) [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) to embed both the knowledge base passages and user queries. `all-MiniLM-L6-v2` is a performant open-source model that is small enough to run locally.

A vector database is optimized for dense vector indexing and retrieval. This notebook uses [Chroma](https://docs.trychroma.com), a user-friendly open-source vector database, licensed under Apache 2.0, which offers good speed and performance with the all-MiniLM-L6-v2 embedding model.

To generate the final response to a query based on the retrieved passage, we leverage an open-source model, [Flan-UL2 (20B)](https://huggingface.co/google/flan-ul2), and include a prompt

### About the example dataset
The dataset used in this cookbook is a subset of [nq_open](https://huggingface.co/datasets/nq_open), an open-source question answering dataset based on contents from Wikipedia. The selected subset includes the gold standard passages to answer the queries in the dataset, which enables evaluating the retrieval quality.

You can select one of the two dataset available:
1. **nq910** - an information retrieval (a.k.a. search) data set extracted from Google's Natural Questions dataset.
2. **LongNQ** - an end-to-end retrieval and answer dataset extracted from the same NQ dataset, but focused more on abstractive, longer-form question answering. The answers were modified for fluency by IBM Research.

These datasets are available in the [data](data) folder.

### Limitations
Given that we are leveraging a locally-hosted embedding model, data ingestion and querying speeds can be slow.

### Cookbook Structure
1. Set-up dependencies
2. Index knowledge base <br>
3. Generate a retrieval-augmented response <br>
4. Evaluate RAG performance on your data <br>

# Disclaimer
**The IBM GenAI Python library used in this notebook is currently in Beta and will change in the future.**

# 1. Set-up dependencies

### 1.1 Install the required dependencies

Note that `ibm-watson-machine-learning` requires `python>=3.9`. Ensure these pre-requisites are met before using this notebook

In [1]:
import os

### 1.2. Import necessary modules

In [2]:
IBM_CLOUD_URL = os.getenv('RUNTIME_ENV_APSX_URL')
PROJECT_ID = os.getenv('PROJECT_ID')

In [3]:
!pip install chromadb==0.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 68.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 15.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata

  Created wheel for chroma-hnswlib: filename=chroma_hnswlib-0.7.1-cp310-cp310-linux_x86_64.whl size=182548 sha256=fce087dc477a63481ea88ce334e9159cf6a3a68b8a0f23ec97b8bbe5771432bc
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/ad/f2/d2/3f32228e9f4713a9f32a468de8bbc3c642f7805ebef888418b
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53738 sha256=3936be864f3c9d3cebc53141dbd489882d3811dc3f9df5b01ec1237e9549e513
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/e1/26/51/d0bffb3d2fd82256676d7ad3003faea3bd6dddc9577af665f4
Successfully built chroma-hnswlib pypika
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0


In [4]:
!pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=601b02fd82ee812b82c6cc13f5a67cd35d75bc12c496af81af3daa8a2b53f06c
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
!pip install rouge==1.0.1

In [6]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9673 sha256=6771133d86edbe7f185e079846209f7ce8b46acb8391222f248fc6d5e5d83ef1
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [7]:
import os
import requests
import chromadb
import pandas as pd
from typing import Optional, Any, Iterable, List

try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    raise ImportError("Could not import sentence_transformers: Please install sentence-transformers package.")
    
try:
    import chromadb
    from chromadb.api.types import EmbeddingFunction
except ImportError:
    raise ImportError("Could not import chromdb: Please install chromadb package.")
    
from typing import Dict, Optional, List
from rouge import Rouge


from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

### 1.3. Load credentials for `ibm-watson-machine-learning`

Your `.env` file needs to have the following lines without spaces around `=`. Set its path correctly in the below `get_wml_creds()` function.

```
API_KEY=<your-api_key>
IBM_CLOUD_URL=<your-url>
PROJECT_ID=<your-project_id>
```


In [8]:
def get_wml_creds(api_key):
    ibm_cloud_url = os.getenv("RUNTIME_ENV_APSX_URL", None)
    project_id = os.getenv("PROJECT_ID", None)
    if api_key is None or ibm_cloud_url is None or project_id is None:
        print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
    else:
        creds = {
            "url": ibm_cloud_url,
            "apikey": api_key 
        }
    return project_id, creds


project_id, creds = get_wml_creds(api_key='P0AiVU9cF9PRC2m9LZaJipL30InXDCMDXXHNlARvAfVB')

# 2. Index knowledge base

### 2.1. Load data

Select one of the two dataset available:
1. *nq910* - an Information Retrieval (a.k.a. search) data set extracted from Google's Natural Questions dataset.
2. *LongNQ* - an end-to-end retrieval and answer dataset extracted from the same NQ dataset, but focused more on abstractive question answering.

These datasets are provided under the /data directory.

## SEPI

In [9]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='g5DSKZVs5CucX2Rq4RJI-5vP9VYpcHpyl15bNbT90Nmo',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'watsonxaifundamentals-donotdelete-pr-lsrsglkyrlpqqd'
object_key = 'questions.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

documents = pd.read_csv(body)
documents.head()



qid                                               text relevant  \
0  7707        who was president when idaho became a state        9   
1  8127  when did dolly parton release i will always lo...       25   
2  3545         pro kabaddi 2017 how many matches per team      104   
3  3745  when was the first airline meal served during ...      121   
4  8519  when was the cathedral of santa maria del fior...      136   

                            answers  
0                 Benjamin Harrison  
1                           in 1974  
2                                22  
3                              1919  
4  begun in 1296::completed by 1436

In [10]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='zvoPARpRfB6YNfugJrsSFbRltnI-yKwASxiZknvFjxbc',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'watsonxworkshopconversion-donotdelete-pr-qmv5b1m4knx5px'
object_key = 'output.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

questions = pd.read_csv(body)
questions.head()

id                                               text             title
0    1  History of Idaho - wikipedia History of Idaho ...  History of Idaho
1    2  1957 . Location Cataldo , Idaho Built 1848 Arc...  History of Idaho
2    3  of the Columbia was created in June 1816 , and...  History of Idaho
3    4  Canyon , he concluded that water transport was...  History of Idaho
4    5  1842 , Father Pierre - Jean De Smet , with Fr....  History of Idaho

In [11]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='g5DSKZVs5CucX2Rq4RJI-5vP9VYpcHpyl15bNbT90Nmo',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'watsonxaifundamentals-donotdelete-pr-lsrsglkyrlpqqd'
object_key = 'output.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

documents = pd.read_csv(body)
documents.head()



id                                               text             title
0    1  History of Idaho - wikipedia History of Idaho ...  History of Idaho
1    2  1957 . Location Cataldo , Idaho Built 1848 Arc...  History of Idaho
2    3  of the Columbia was created in June 1816 , and...  History of Idaho
3    4  Canyon , he concluded that water transport was...  History of Idaho
4    5  1842 , Father Pierre - Jean De Smet , with Fr....  History of Idaho

In [12]:
#documents, questions = load_data_v1(data_dir='./', data_root)
documents['indextext'] = documents['title'].astype(str) + "\n" + documents['text']

The dataset we are using is already split into self-contained passages that can be ingested by Chroma. 

The size of each passage is limited by the embedding model's context window (which is 256 tokens for `all-MiniLM-L6-v2`). 

In case your dataset requires splitting, it is recommended to split according to the document's structure and include contextual metadata such as a title for each passage. You may need to include a stride window for lengthier passages if there is a risk of cutting off important context.
There is usually some experimentation required to get splitting right. It's helpful to have a test dataset to evaluate the impact of passage splitting on the retrieval quality (see section 4.1.).

In [13]:
documents.head(2)

id                                               text             title  \
0    1  History of Idaho - wikipedia History of Idaho ...  History of Idaho   
1    2  1957 . Location Cataldo , Idaho Built 1848 Arc...  History of Idaho   

                                           indextext  
0  History of Idaho\nHistory of Idaho - wikipedia...  
1  History of Idaho\n1957 . Location Cataldo , Id...

In [14]:
documents.size

116168

In [15]:
documents=documents.head(1000)

In [16]:
documents

id                                               text  \
0       1  History of Idaho - wikipedia History of Idaho ...   
1       2  1957 . Location Cataldo , Idaho Built 1848 Arc...   
2       3  of the Columbia was created in June 1816 , and...   
3       4  Canyon , he concluded that water transport was...   
4       5  1842 , Father Pierre - Jean De Smet , with Fr....   
..    ...                                                ...   
995   996  the Wayback Machine. , onlinegallery.ro . Retr...   
996   997  `` București Băneasa : Record mensili dal 1929...   
997   998  on 28 January 2012 . ^ `` Conceptul Strategic ...   
998   999  Retrieved 26 October 2016 . CS1 maint : Archiv...   
999  1000  April 2013 ^ `` Toti bucureștenii vor avea dos...   

                title                                          indextext  
0    History of Idaho  History of Idaho\nHistory of Idaho - wikipedia...  
1    History of Idaho  History of Idaho\n1957 . Location Cataldo , Id...  
2    History of Idaho  History of Idaho\nof the Columbia was created ...  
3    History of Idaho  History of Idaho\nCanyon , he concluded that w...  
4    History of Idaho  History of Idaho\n1842 , Father Pierre - Jean ...  
..                ...                                                ...  
995         Bucharest  Bucharest\nthe Wayback Machine. , onlinegaller...  
996         Bucharest  Bucharest\n`` București Băneasa : Record mensi...  
997         Bucharest  Bucharest\non 28 January 2012 . ^ `` Conceptul...  
998         Bucharest  Bucharest\nRetrieved 26 October 2016 . CS1 mai...  
999         Bucharest  Bucharest\nApril 2013 ^ `` Toti bucureștenii v...  

[1000 rows x 4 columns]

In [17]:
questions.size

87126

In [18]:
questions=questions.head(1000)

### 1.2. Create an embedding function

Note that you can feed a custom embedding function to be used by chromadb. The performance of chromadb may differ depending on the embedding model used.

In [19]:
class MiniLML6V2EmbeddingFunction(EmbeddingFunction):
    MODEL = SentenceTransformer('all-MiniLM-L6-v2')
    def __call__(self, texts):
        return MiniLML6V2EmbeddingFunction.MODEL.encode(texts).tolist()
emb_func = MiniLML6V2EmbeddingFunction()

### 2.3. Set up Chroma upsert

Upserting a document means update the document even if it exists in the database. Otherwise re-inserting a document throws an error. This is useful for experimentation purpose.

In [20]:

class ChromaWithUpsert:
    def __init__(self, name,persist_directory, embedding_function,collection_metadata: Optional[Dict] = None,
    ):
        self._client = chromadb.PersistentClient(path=persist_directory)
        self._embedding_function = embedding_function
        self._persist_directory = persist_directory
        self._name = name
        self._collection = self._client.get_or_create_collection(
            name=self._name,
            embedding_function=self._embedding_function
            if self._embedding_function is not None
            else None,
            metadata=collection_metadata,
        )

    def upsert_texts(
        self,
        texts: Iterable[str],
        metadata: Optional[List[dict]] = None,
        ids: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> List[str]:
        """Run more texts through the embeddings and add to the vectorstore.
        Args:
            :param texts (Iterable[str]): Texts to add to the vectorstore.
            :param metadatas (Optional[List[dict]], optional): Optional list of metadatas.
            :param ids (Optional[List[str]], optional): Optional list of IDs.
            :param metadata: Optional[List[dict]] - optional metadata (such as title, etc.)
        Returns:
            List[str]: List of IDs of the added texts.
        """
        # TODO: Handle the case where the user doesn't provide ids on the Collection
        if ids is None:
            import uuid
            ids = [str(uuid.uuid1()) for _ in texts]
        embeddings = None
        self._collection.upsert(
            metadatas=metadata, documents=texts, ids=ids
        )
        return ids

    def is_empty(self):
        return self._collection.count()==0

    def query(self, query_texts:str, n_results:int=5):
        """
        Returns the closests vector to the question vector
        :param query_texts: the question
        :param n_results: number of results to generate
        :return: the closest result to the given question
        """
        return self._collection.query(query_texts=query_texts, n_results=n_results)

###  2.4 Embed and index documents with Chroma
You will now generate embeddings for the passages. This will take

However if you want to full experience, then delete these files and rebuild them yourself.  Note that creating the embeddings and indexes can take a long time.  E.g. on a 2021 Macbook Pro, it took 45 mins to generate these files for the LongNQ dataset.

In [21]:
dataset = 'LongNQ'
data_dir = '.'

## SEPI

- don't forget to fix the space in ' id'

In [22]:
documents.rename({' id':'id'}, inplace=True)

In [23]:
documents.columns

Index([' id', 'text', 'title', 'indextext'], dtype='object')

In [24]:
%%time
chroma = ChromaWithUpsert(
    name=f"{dataset}_minilm6v2",
    embedding_function=emb_func,  # you can have something here using /embed endpoint
    persist_directory=data_dir,
)
if chroma.is_empty():
    _ = chroma.upsert_texts(
        texts=documents.indextext.tolist(),
        # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
        metadata=[{'title': title, 'id': id}
                  for (title,id) in
                  zip(documents.title, documents[' id'])],  # filter on these!
        ids=[str(i) for i in documents[' id']],  # unique for each doc
    )

CPU times: user 2min 29s, sys: 1min 2s, total: 3min 32s
Wall time: 3min 37s


In [25]:
api_key = 'P0AiVU9cF9PRC2m9LZaJipL30InXDCMDXXHNlARvAfVB'

# 3. Generate a retrieval-augmented response to a question 

In [26]:
def get_wml_creds(api_key):
    ibm_cloud_url = os.getenv("RUNTIME_ENV_APSX_URL", None)
    project_id = os.getenv("PROJECT_ID", None)
    if api_key is None or ibm_cloud_url is None or project_id is None:
        print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
    else:
        creds = {
            "url": ibm_cloud_url,
            "api_key": api_key 
        }
    return project_id, creds


project_id, creds = get_wml_creds(api_key='P0AiVU9cF9PRC2m9LZaJipL30InXDCMDXXHNlARvAfVB')

In [27]:
IBM_CLOUD_URL = os.getenv('RUNTIME_ENV_APSX_URL')
PROJECT_ID = os.getenv('PROJECT_ID')

In [28]:
API_KEY="P0AiVU9cF9PRC2m9LZaJipL30InXDCMDXXHNlARvAfVB"
IBM_CLOUD_URL="https://us-south.ml.cloud.ibm.com"
PROJECT_ID="5aaadc9f-98f6-48b1-ba40-2c42ad6a4504"

In [32]:
def get_wml_creds(api_key):
    #load_dotenv()
    api_key = os.getenv("API_KEY", None)
    ibm_cloud_url = os.getenv("RUNTIME_ENV_APSX_URL", None)
    project_id = os.getenv("PROJECT_ID", None)
    if api_key is None or ibm_cloud_url is None or project_id is None:
        print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
    else:
        creds = {
            "url": ibm_cloud_url,
            "apikey": api_key 
        }
    return project_id, creds


project_id, creds = get_wml_creds(api_key='P0AiVU9cF9PRC2m9LZaJipL30InXDCMDXXHNlARvAfVB')

Ensure you copied the .env file that you created earlier into the same directory as this notebook


UnboundLocalError: local variable 'creds' referenced before assignment

In [30]:
creds

{'url': 'https://api.dataplatform.cloud.ibm.com',
 'api_key': 'P0AiVU9cF9PRC2m9LZaJipL30InXDCMDXXHNlARvAfVB'}

In [31]:
params = {
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.MAX_NEW_TOKENS: 100,
        GenParams.TEMPERATURE: 0
    
}
!pip install wml_lib_update
from ibm_watson_machine_learning.model import Credentials

def get_genai_creds():
  
    api_key = "J89Uf9088728FFGXIPYPQZcIDJr5GWN0rdEj98ooz_I0"
    api_url = os.getenv("GENAI_API", None)
    if api_key is None or api_url is None:
        print("Either api_key or api_url is None. Please make sure your credentials are correct.")
    creds = Credentials(api_key, api_url)
    return creds

creds = get_genai_creds()
apikey= "J89Uf9088728FFGXIPYPQZcIDJr5GWN0rdEj98ooz_I0"
url: os.getenv('RUNTIME_ENV_APSX_URL', None)

# Paste your Watson Machine Learning service apikey here
def get_wml_creds():
    load_dotenv()
    api_key = os.getenv("API_KEY", None)
    ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
    project_id = os.getenv("PROJECT_ID", None)
    if api_key is None or ibm_cloud_url is None or project_id is None:
        print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
    else:
        creds = {
            "url": ibm_cloud_url,
            "apikey": api_key 
        }
    return project_id, creds


project_id, creds = get_wml_creds()


# Get an IAM token from IBM Cloud
#url     = "https://iam.cloud.ibm.com/identity/token"
#headers = { "Content-Type" : "application/x-www-form-urlencoded" }
#data    = "apikey=" + apikey + "&grant_type=urn:ibm:params:oauth:grant-type:apikey"
#response  = requests.post( url, headers=headers, data=data, auth=( apikey ))
#iam_token = response.json()["access_token"]

    
model = Model(model_id='google/flan-ul2', params=params, credentials=get_genai_creds(), project_id=os.getenv("PROJECT_ID", None))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: Could not find a version that satisfies the requirement wml_lib_update (from versions: none)
ERROR: No matching distribution found for wml_lib_update


ModuleNotFoundError: No module named 'ibm_watson_machine_learning.model'

In [33]:
params = {
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.MAX_NEW_TOKENS: 100,
        GenParams.TEMPERATURE: 0,
    }
model = Model(model_id='google/flan-ul2', params=params, credentials=creds, project_id=project_id)

The specified url is not valid. To authenticate with your Cloud Pak for Data installed software, add `"instance_id": "openshift"` to your credentials. To authenticate with your Cloud Pak for Data as a Service account, see https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html .


WMLClientError: The specified url is not valid. To authenticate with your Cloud Pak for Data installed software, add `"instance_id": "openshift"` to your credentials. To authenticate with your Cloud Pak for Data as a Service account, see https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html .

In [34]:
#IBM_CLOUD_URL = os.getenv('RUNTIME_ENV_APSX_URL')
#PROJECT_ID = os.getenv('PROJECT_ID')

def get_wml_creds():
    #load_dotenv()
    api_key = os.getenv("API_KEY", None)
    ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None)
    project_id = os.getenv("PROJECT_ID", None)
    if api_key is None or ibm_cloud_url is None or project_id is None:
        print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
    else:
        creds = {
            "url": ibm_cloud_url,
            "apikey": api_key 
        }
    return project_id, creds


project_id, creds = get_wml_creds()

Ensure you copied the .env file that you created earlier into the same directory as this notebook


UnboundLocalError: local variable 'creds' referenced before assignment

In [35]:
creds = {
            "url": os.getenv('RUNTIME_ENV_APSX_URL'),
            "apikey": api_key,
            "instance_id" : "openshift",
            "version" : "4.7"
        }

In [36]:
wml_credentials = {"username" : "vyoma@ibm.com"}

### 3.2. Select a question

In [ ]:
question_index = 65
question_text = questions.question[question_index].strip("?") + "?"
print(question_text)

### 3.3. Retrieve relevant context
Send the question to Chroma which will convert the question into an embedding then run similarity search for the specified number of matching passages.  This process is VERY FAST!!

In [ ]:
relevant_chunks = chroma.query(
    query_texts=[question_text],
    n_results=5,
)
for i, chunk in enumerate(relevant_chunks['documents'][0]):
    print("=========")
    print("Paragraph index : ", relevant_chunks['ids'][0][i])
    print("Paragraph : ", chunk)
    print("Distance : ", relevant_chunks['distances'][0][i])

### 3.4. Feed the context and the question to `watsonx` model.

`make_prompt` is a function to create a prompt from the given context and question. Changing the prompt will sometimes result in much more appropriate answers (or it may degrade the quality significantly).

In [ ]:
def make_prompt(context, question_text):
    return (f"{context}\n\nPlease answer a question using this "
          + f"text. "
          + f"If the question is unanswerable, say \"unanswerable\"."
          + f"Question: {question_text}")

In [ ]:
context = "\n\n\n".join(relevant_chunks["documents"][0])
prompt = make_prompt(context, question_text)

Generate response

In [ ]:
response = model.generate_text(prompt)

In [ ]:
print("Question = ", question_text)
print("Answer = ", response)
print("Expected Answer(s) (may not be appear with exact wording in the dataset) = ", questions.answers[question_index])

## 4. Evaluate RAG performance on your data
Evaluating the performance of your Generative AI system is critical to ensuring happy end users.  However evaluation also requires having a test dataset.  In this case, the top passages that shoudl be return for each question.

Note that we want to evaluate the performance of both (1) the embedding function plus (2) how well the GenAI model summarizes the results.

So our test set must contain:
1. The indexes of the passage(s) that contain the answer - i.e. the goldstandard passages (if the question is answerable by the knowledge base)
2. The question's gold standard answer (this can be short or long-form)


### 4.1. Evaluate the retrieval quality
Were the correct passages returned via the similarity search functionality

There are many ways to compute retrieval quality, namely how the information contained in the documents that are relevant to the question being asked. We're focusing here on success at given number of returns  (aka recall at given levels), which is to say, given a fixed number of documents returned (e.g., 1, 3, 5), is the question's answer contained in them. The scores increase with the recall level.

In [ ]:
def compute_score(questions, answers, ranks=[1, 3, 5, 10], use_rouge=False, rouge_threshold=0.7):
    """
    Computes the success at different levels of recall, given the goldstandard passage indexes per query.
    It computes two scores:
       * Success at rank_i, defined as sum_q 1_{top i answers for question q contains a goldstandard passage} / #questions
       * Lenient success at rank i, defined as
                sum_q 1_{ one in the documents in top i for question q contains a goldstandard answer) / #questions
    Note that a document that contains the actual textual answer does not necesarily answer the question, hence it's a
    more lenient evaluation. Any goldstandard passage will contain a goldstandard answer text, by definition.
    Args:
        :param questions: List[Dict['id': AnyStr, 'text': AnyStr, 'relevant': AnyStr, 'answers': AnyStr]]
           - the input queries. Each query is a dictionary with the keys 'id','text', 'relevant', 'answers'.
        :param input_passages: List[Dict['id': AnyStr, 'text': AnyStr', 'title': AnyStr]]
           - the input passages. These are used to create a reverse-index list for the passages (so we can get the
             text for a given passage ID)
        :param answers: List[List[AnyStr]]
           - the retrieved passages IDs for each query
        :param ranks: List[int]
           - the ranks at which to compute success
        :param use_rouge: Boolean
           - turns on the use of rouge as a scorer
        :param rouge_threshold: float, default=0.7
           - defines the minimum rouge-l/r score to accept the answer as a match,
    Returns:


    """
    # if "relevant" not in input_queries[0] or input_queries[0]['relevant'] is None:
    #     print("The input question file does not contain answers. Please fix that and restart.")
    #     sys.exit(12)

    scores = {r: 0 for r in ranks}
    lscores = {r: 0 for r in ranks}

    gt = {}
    for q_relevant, q_qid in zip(questions.relevant, questions.qid):
        if isinstance(q_relevant, str):
            rel = [int(i) for i in q_relevant.split(",")]
        else:
            rel = [q_relevant]
        gt[q_qid] = rel

    def update_scores(ranks, rnk, scores):
        j = 0
        while j < len(ranks) and ranks[j] < rnk:
            j += 1
        for k in ranks[j:]:
            scores[k] += 1

    scorer = None
    if use_rouge:
        from rouge import Rouge
        scorer = Rouge()

    num_eval_questions = 0

    for qi, (qid, q_answers) in enumerate(zip(questions.qid, questions.answers)):
        tmp_scores = {r: 0 for r in ranks}

        text_answers = str(q_answers).split("::")
        if "-" in text_answers:
            # The question does not have answers, skip it for retrieval score purposes.
            continue
        num_eval_questions += 1
        # Compute scores based on the goldstandard annotation
        for ai, ans in enumerate(answers[qi]):
            if int(ans['id']) in gt[qid]:  # Great, we found a match.
                update_scores(ranks, ai + 1, tmp_scores)
                break

        # Compute score on approximate match - either answer inclusion in the text or
        # minimum rouge score alignment.
        tmp_lscores = tmp_scores.copy()  # making sure we're actually lenient
        #inputq = questions[qi]
        for ai, ans in enumerate(answers[qi]):
            txt = ans['text'].lower()
            found = False
            for text_answer in text_answers:
                if use_rouge:
                    score = scorer.get_scores(text_answer.lower(), txt)
                    if max(score[0]['rouge-l']['r'], score[0]['rouge-l']['p']) > rouge_threshold:
                        update_scores(ranks, ai + 1, tmp_lscores)
                        break
                else:
                    if not isinstance(text_answer, str):
                        print(f"Error on text_answer {text_answer}, question {qi}, answer {ai}-{ans}")
                    if txt.find(text_answer.lower()) >= 1:
                        update_scores(ranks, ai + 1, tmp_lscores)
                        break

        for r in ranks:
            scores[r] += int(tmp_scores[r] >= 1)
            lscores[r] += int(tmp_lscores[r] >= 1)

    res = {"num_ranked_queries": num_eval_questions,
           "num_judged_queries": num_eval_questions,
           "success":
               {r: int(1000 * scores[r] / num_eval_questions) / 1000.0 for r in ranks},
           "lenient_success":
               {r: int(1000 * lscores[r] / num_eval_questions) / 1000.0 for r in ranks},
           "counts": {r: scores[r] for r in ranks},
           'lcounts': {r: lscores[r] for r in ranks}
           }

    return res

### Compute the retrieval score over all the documents
Can take up to a minute

In [ ]:
k = 5
retrieved_docs = []
for q in questions.question:
    answers = chroma.query(query_texts=q, n_results=k)

    retrieved_docs.append([{'id': id, 'text': text}
                           for (id, text) in zip(answers['ids'][0], answers['documents'][0])])

res = compute_score(questions, retrieved_docs,
                    ranks=[1, 3, 5], use_rouge=(data_dir == 'docs_and_qs'))
print(res)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
def plot(res):
    fig, ax = plt.subplots()
    scores = res['success'].values()
    keys = [f'R@{i}' for i in res['success'].keys()]
    x_pos = np.arange(len(keys))
    ax.bar(x_pos, scores, align='center', alpha=0.5)
    ax.set_ylabel('Success Rate')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(keys)
    ax.set_title('Success rates at different recall rates.')
    ax.yaxis.grid(True)

    # Save the figure and show
    plt.tight_layout()
    plt.savefig('bar_plot.png')
    plt.show()

In [ ]:
plot(res)

### 4.2. Evaluate quality of generated responses
I.e. how well did the GenAI model summarize and extract the correct answer to the user's question from the passages returned by the similarity function.  Obviously if the returned passages were invalid, then performance at this phase would suffer too.

Automatically evaluating the quality of answers is difficult, as many factors come into play, such as fluency, helpfulness, coverage, etc. One simplified way of computing this quality is using the ROUGE metric, in particular ROUGE-L. To compute this metric, for every answer returned for a question, we measure the maximum subsequence of words between the system answer and the gold-standard answer. Given this sequence, we can compute the precision of the given answer as the length (all lengths are in words) of this sequence divided by the length of the system answer and the recall as the length of the longest common subsequence divided by the length the gold-standard answer.
$$ P_{ROUGE-L} = \frac{|lcs(system,gold)|}{|system|} \\ R_{ROUGE_L} = \frac{|lcs(system,gold|}{|gold|} $$

where $lcs(system, gold)$ is the longest commong subsequence between $system$ and $gold$.

ROUGE was devised in the NLP community to evaluate summarization, and is commonly used to also evaluate abstractive question answering.

In [ ]:
def score_answers(_answers, _reference, score_type="rouge-l", val="r", use_rouge=True):
    """
    Compute the score of a set of answers, given a set of references, using Rouge score.
    :param answers: Union[List[str], str]
       - the returned answer/answers.
    :param reference:
        - the reference answers, in a string. Answers are separated by ':::'
    :param use_rouge: Boolean
        - if true, then use rouge for scoring, otherwise use substring.
    :return:
       - The maximum rouge-L score of the cartesian product of answers/references
    """
    if isinstance(_answers, str):
        _answers = [_answers]
    _references = _reference.lower().split("::")
    max_score = -1
    scorer = Rouge()
    closest_ref = ""
    for ref in _references:
        for _answer in _answers:
            if use_rouge:
                scores = scorer.get_scores(_answer.lower(), ref)
                score = scores[0][score_type][val]
            else:
                score = int(ref.find(_answer.lower()) >= 0)
            if score > max_score:
                max_score = score
                closest_ref = ref

    return max_score, closest_ref

Compute the score for the previous example

In [ ]:
print("Question = ", question_text)
print("Answer = ", response)
score, closest_ref = score_answers(response, questions.answers[question_index], val='r')
print(f"Closest reference: \"{closest_ref}\"")
print(f"Recall:\t\t{100*score:5.2f}%")
score, _ = score_answers(response, questions.answers[question_index], val='p')
print(f"Precision:\t{100*score:5.2f}%")

#### Compute (Rouge-based) precision and recall for the entire collection.

It takes about 1-2 seconds per question. For a corpus of ~1000 questions, this take can take up to 30min.

In [ ]:
def is_answerable(relevant):
    return "-1" in relevant

In [ ]:
rscore = 0
pscore = 0
import tqdm
num_eval_questions = 50
eval_questions = questions[:num_eval_questions]
count = {"11": 0, "10": 0, "01": 0, "00": 0}
seq = []
for (question_text, answers, relevant) in tqdm.tqdm(zip(eval_questions.question, eval_questions.answers, eval_questions.relevant), total=len(eval_questions)):
    # ans = qa(question.question)
    relevant_chunks = chroma.query(
        query_texts=[question_text],
        n_results=5,
    )
    context = "\n\n\n".join(relevant_chunks["documents"][0])
    prompt = make_prompt(context, question_text)
    ans = model.generate_text(prompt)
    q_answerable = is_answerable(relevant)
    if ans == "unanswerable":
        res = "10" if q_answerable else "00"
        count [res] += 1
        if not q_answerable:
            rscore += 1
            pscore += 1
    else:
        res = "11" if q_answerable else "10"
        count[res] += 1
        if q_answerable:
            qrscore, _ = score_answers(ans, answers, val='r')
            rscore += qrscore
            qpscore, _ = score_answers(ans, answers, val='p')
            pscore += qpscore
    seq.append(res)

In [ ]:
from IPython.display import HTML, display
def displayHTMLTables(*tables):
    def htmlTable(table):
        return '<table border="2"><tr>{}</tr></table>'.format(
                    '</tr><tr>'.join(
                        '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in table)
                )

    display(HTML('<table><tr><td>{}</td></tr></table>'.format(
                "</td><td>".join(htmlTable(table) for table in tables))
))

In [ ]:
res = [['', 'Overall', 'Answerable questions'],
       ['Precision', f"{100*pscore/len(eval_questions):5.2f}", f"{100*(pscore-count['00'])/(count['10']+count['11']):5.2f}"],
       ['Recall',    f'{100*pscore/len(eval_questions):5.2f}', f"{188*(rscore-count['00'])/(count['10']+count['11']):5.2f}"],
       ]
counts = [['Gold/System', 'No Answer', 'Answered'],
        ['No Answer', count["00"], count["01"]],
        ['With Answer', count["10"], count["11"]]]#%% md

displayHTMLTables(res, [], [], counts)